In [7]:
import pandas as pd
import requests
import pymongo
from sqlalchemy import create_engine



url = 'https://data.cityofchicago.org/api/views/5udb-dr6f/rows.json?accessType=DOWNLOAD'
r= requests.get(url).json()


In [8]:
df = pd.DataFrame(r['data'])
df = df.drop(df.columns[[0, 1, 2, 3, 5, 7]], axis=1, inplace=False)
df =df.dropna(axis=1, how='all', thresh=None, subset=None, inplace=False)
df.rename(columns={8:'Name',9:'DBA',10:'ID', 11:'Type',12:'Risk', 13:'Street',14:'City', 15:'State', 16:'Zip', 17:'Date',18:'Inspection',19:'Success',20:'Description', 21:'lat', 22:'lng'}, inplace=True)
print(df.describe())
df.head(5)

          Name     DBA       ID        Type           Risk  \
count   114427  113425   114427      114427         114425   
unique   13546   12981    18772           1              4   
top     SUBWAY  SUBWAY  1354323  Restaurant  Risk 1 (High)   
freq      2389    2900      135      114427          92311   

                    Street     City   State     Zip                 Date  \
count               114427   114345  114406  114394               114427   
unique                9718        9       1      59                 2105   
top     11601 W TOUHY AVE   CHICAGO      IL   60614  2013-11-14T00:00:00   
freq                  2033   114108  114406    5279                  118   

       Inspection Success                                        Description  \
count      114426  114427                                              86776   
unique         44       7                                              86350   
top       Canvass    Pass  32. FOOD AND NON-FOOD CONTACT SURFACES PR

,Name,DBA,ID,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,lat,lng
0,7-ELEVEN,7-ELEVEN,1933682,Restaurant,Risk 2 (Medium),714 N CLARK ST,CHICAGO,IL,60654,2011-08-18T00:00:00,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.895375850474665,-87.63136270867454
1,CAFE SOCIETY INC,CAFE SOCIETY INC,1171329,Restaurant,Risk 1 (High),1801 S INDIANA AVE,CHICAGO,IL,60616,2011-08-15T00:00:00,Canvass,Fail,24. DISH WASHING FACILITIES: PROPERLY DESIGNED...,41.85783824203235,-87.62218405329706
2,YUMMY YUMMY ASIAN CUISINE,YUMMY YUMMY ASIAN CUISINE,3669,Restaurant,Risk 1 (High),2901 N BROADWAY,CHICAGO,IL,60657,2011-08-15T00:00:00,Complaint Re-Inspection,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.93527996155769,-87.64409274536888
3,SOPO,SOPO,1299591,Restaurant,Risk 1 (High),3418 N SOUTHPORT AVE,CHICAGO,IL,60657,2011-08-16T00:00:00,Complaint Re-Inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.943990250918624,-87.6640799966287
4,GT's Fast Food,GT's Fast Food,2014164,Restaurant,Risk 1 (High),5016 N Pulaski RD,CHICAGO,IL,60630,2011-08-16T00:00:00,Canvass Re-Inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.97240585532906,-87.72829150056414


In [112]:
###########Method 1 MONGO DB   #################

#insert into Mongo
#REMEMBER TO START MONGOD in your bash!!!!!!!!
conn = 'mongodb://localhost:27017'
client=pymongo.MongoClient(conn)
#define DB and collection/table to work with
db = client.ChicagoRest
#convert DataFrame to JSON format for insert
#Load all dataframes
records = json.loads(df.to_json(orient='records'))
db.CompanyData.insert_many(records)

In [14]:
#sqlite
connection_string = "sqlite:///ChicagoRestaurants.sqlite"
engine = create_engine(connection_string)
#list tables names- if you want
#engine.table_names()

####Create the database to match our columns in our dataframe####
#Name	DBA	ID	Type	Risk	Street	City	State	Zip	Date	Inspection	Success	Description	lat	lng


##############IMPORTANT###############################
########### hash out this after creating as it will give an error stating the DB already exists after running the first time ####
engine.execute("CREATE TABLE RestaurantData(id INT PRIMARY KEY, Name TEXT, DBA TEXT, ID2 TEXT, Type TEXT, Risk TEXT, Street TEXT, City TEXT, State TEXT, Zip TEXT, Date TEXT, Inspection TEXT, Success TEXT, Description TEXT, Lat Int, Lng Int);")
################################################################################################################

####### PLUG that data into SQLITE#####
df.to_sql(name="ChicagoRestaurants", con=engine, if_exists="append", index=False)
#####SHOW ME THE MONEY#####
pd.read_sql_query("SELECT * FROM ChicagoRestaurants",con=engine).head()

,Name,DBA,ID,Type,Risk,Street,City,State,Zip,Date,Inspection,Success,Description,lat,lng
0,7-ELEVEN,7-ELEVEN,1933682,Restaurant,Risk 2 (Medium),714 N CLARK ST,CHICAGO,IL,60654,2011-08-18T00:00:00,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.895375850474665,-87.63136270867454
1,CAFE SOCIETY INC,CAFE SOCIETY INC,1171329,Restaurant,Risk 1 (High),1801 S INDIANA AVE,CHICAGO,IL,60616,2011-08-15T00:00:00,Canvass,Fail,24. DISH WASHING FACILITIES: PROPERLY DESIGNED...,41.85783824203235,-87.62218405329706
2,YUMMY YUMMY ASIAN CUISINE,YUMMY YUMMY ASIAN CUISINE,3669,Restaurant,Risk 1 (High),2901 N BROADWAY,CHICAGO,IL,60657,2011-08-15T00:00:00,Complaint Re-Inspection,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.93527996155769,-87.64409274536888
3,SOPO,SOPO,1299591,Restaurant,Risk 1 (High),3418 N SOUTHPORT AVE,CHICAGO,IL,60657,2011-08-16T00:00:00,Complaint Re-Inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.943990250918624,-87.6640799966287
4,GT's Fast Food,GT's Fast Food,2014164,Restaurant,Risk 1 (High),5016 N Pulaski RD,CHICAGO,IL,60630,2011-08-16T00:00:00,Canvass Re-Inspection,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.97240585532906,-87.72829150056414
